# LangChain EnumOutputParser 예제: 감정 분석
`ChatOpenAI`와 `EnumOutputParser`를 사용해 텍스트의 감정을 **긍정/부정/중립**으로 분류합니다.

### 오류 해결 포인트
- 더 강력하고 명확한 프롬프트 작성
- OutputFixingParser 추가로 안정성 향상
- 에러 처리 로직 포함

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import EnumOutputParser, OutputFixingParser
from langchain.schema import OutputParserException

from enum import Enum
from pprint import pprint


In [2]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_f


In [3]:
# 감정 클래스 정의 (Enum)
class Sentiment(str, Enum):
    POSITIVE = "긍정"
    NEGATIVE = "부정"
    NEUTRAL = "중립"

In [4]:
# EnumOutputParser 초기화
parser = EnumOutputParser(enum=Sentiment)
format_instructions = parser.get_format_instructions()

print("감정 분류 출력 형식:")
print(format_instructions)

감정 분류 출력 형식:
Select one of the following options: 긍정, 부정, 중립


In [5]:
# 프롬프트 템플릿
template = """
당신은 텍스트 감정 분석 전문가입니다.
다음 텍스트의 감정을 분석하고, 반드시 아래 세 가지 중 하나의 단어로만 답변하세요.

텍스트: "{text}"

{format_instructions}

중요 규칙:
1. 반드시 "긍정", "부정", "중립" 중 하나의 단어만 출력하세요.
2. 다른 설명이나 부가 설명을 추가하지 마세요.
3. 이모지나 특수문자도 포함하지 마세요.
4. 오직 하나의 단어만 출력하세요.

답변:"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)
print(prompt)

input_variables=['text'] input_types={} partial_variables={'format_instructions': 'Select one of the following options: 긍정, 부정, 중립'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'text'], input_types={}, partial_variables={}, template='\n당신은 텍스트 감정 분석 전문가입니다.\n다음 텍스트의 감정을 분석하고, 반드시 아래 세 가지 중 하나의 단어로만 답변하세요.\n\n텍스트: "{text}"\n\n{format_instructions}\n\n중요 규칙:\n1. 반드시 "긍정", "부정", "중립" 중 하나의 단어만 출력하세요.\n2. 다른 설명이나 부가 설명을 추가하지 마세요.\n3. 이모지나 특수문자도 포함하지 마세요.\n4. 오직 하나의 단어만 출력하세요.\n\n답변:'), additional_kwargs={})]


In [6]:
# ChatOpenAI 모델 초기화
# 환경변수에서 OpenAI API 키 설정 (실제 사용시 주석 해제)
# os.environ["OPENAI_API_KEY"] = "your-api-key"

# model = ChatOpenAI(
#     model="gpt-3.5-turbo",
#     temperature=0  # 일관성을 위해 0으로 설정
# )
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0  # 일관성을 위해 0으로 설정
)

# OutputFixingParser로 안정성 향상
fixing_parser = OutputFixingParser.from_llm(parser=parser, llm=model)
print(fixing_parser)

print("모델 및 파서 설정 완료")

parser=EnumOutputParser(enum=<enum 'Sentiment'>) retry_chain=PromptTemplate(input_variables=['completion', 'error', 'instructions'], input_types={}, partial_variables={}, template='Instructions:\n--------------\n{instructions}\n--------------\nCompletion:\n--------------\n{completion}\n--------------\n\nAbove, the Completion did not satisfy the constraints given in the Instructions.\nError:\n--------------\n{error}\n--------------\n\nPlease try again. Please only respond with an answer that satisfies the constraints laid out in the Instructions:')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001EE73BCEA20>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001EE7383E270>, root_client=<openai.OpenAI object at 0x000001EE720653A0>, root_async_client=<openai.AsyncOpenAI object at 0x000001EE7383E180>, model_name='moonshotai/kimi-k2-instruct-0905', temperature=0.0, model_kwargs={}, openai_api_key=Secre

In [10]:
# 테스트 텍스트
texts = [
    "이 제품 정말 좋아요! 완전 만족스러워요.",
    "서비스가 너무 느리고 불친절했습니다.",
    "이 제품 그런대로 괜찮습니다.",
    "배송은 빠르지만 품질이 아쉽습니다.",
    "최고의 경험이었습니다!",
    "완전 실망했어요... 최악이에요",
    "오늘 오전에는 비가 내렸습니다."
]

print(f"테스트할 텍스트 {len(texts)}개 준비 완료")

테스트할 텍스트 7개 준비 완료


In [8]:
# 수동 테스트 (API 키 없이도 작동)
def manual_test():
    """수동 테스트 - API 키 없이도 확인 가능"""
    print("=== 수동 테스트 (시뮬레이션) ===")
    
    # 올바른 형식의 응답들
    test_responses = ["긍정", "부정", "중립"]
    
    for response in test_responses:
        try:
            parsed = parser.parse(response)
            print(f" '{response}' → {parsed.value} (성공)")
        except Exception as e:
            print(f" '{response}' → 실패: {e}")
    
    # 잘못된 형식의 응답들
    wrong_responses = [
        "분석 결과: 긍정",
        "이 텍스트는 긍정적입니다",
        "positive"
    ]
    
    print("\n잘못된 형식 테스트:")
    for response in wrong_responses:
        try:
            parsed = parser.parse(response)
            print(f" '{response}' → {parsed.value} (예상외 성공)")
        except Exception as e:
            print(f" '{response}' → 파싱 실패 (예상됨)")

# 수동 테스트 실행
manual_test()

=== 수동 테스트 (시뮬레이션) ===
 '긍정' → 긍정 (성공)
 '부정' → 부정 (성공)
 '중립' → 중립 (성공)

잘못된 형식 테스트:
 '분석 결과: 긍정' → 파싱 실패 (예상됨)
 '이 텍스트는 긍정적입니다' → 파싱 실패 (예상됨)
 'positive' → 파싱 실패 (예상됨)


In [11]:
# 안전한 감정 분석 함수 (에러 처리 포함)
def safe_sentiment_analysis(text, use_fixing_parser=True):
    """안전한 감정 분석 함수 - 에러 처리 포함"""
    try:
        # 기본 체인 생성
        # use_fixing_parser가 True이면 OutputFixingParser가 적용되고, False이면 EnumOutputParser가 적용됨
        chain = prompt | model | (fixing_parser if use_fixing_parser else parser)
        
        # 분석 실행
        result = chain.invoke({"text": text})
        return result, None
        
    except OutputParserException as e:
        return None, f"파싱 오류: {str(e)[:100]}..."
    except Exception as e:
        return None, f"일반 오류: {str(e)[:100]}..."

# 실제 감정 분석 실행 (API 키 필요)
def run_sentiment_analysis():
    """실제 감정 분석 실행"""
    print("=== 실제 감정 분석 결과 ===")
    
    success_count = 0
    total_count = len(texts)
    
    for i, text in enumerate(texts, 1):
        print(f"\n{i}. 텍스트: {text}")
        
        # OutputFixingParser 사용
        result, error = safe_sentiment_analysis(text, use_fixing_parser=True)
        
        if result:
            print(f"   감정: {result.value} ")
            success_count += 1
        else:
            print(f"   오류: {error} ")
            
            # 기본 파서로 재시도
            print("   기본 파서로 재시도...")
            result2, error2 = safe_sentiment_analysis(text, use_fixing_parser=False)
            
            if result2:
                print(f"   감정: {result2.value} (기본 파서 성공)")
                success_count += 1
            else:
                print(f"   재시도 실패: {error2} ")
    
    print(f"\n=== 결과 요약 ===")
    print(f"성공: {success_count}/{total_count} ({success_count/total_count*100:.1f}%)")
    print(f"실패: {total_count-success_count}/{total_count}")

# 실제 분석 실행 (API 키가 있는 경우)
try:
    run_sentiment_analysis()
except Exception as e:
    print("API 키가 설정되지 않았거나 네트워크 오류:")
    print("실제 실행을 위해서는 OpenAI API 키를 설정하세요.")
    print(f"오류 상세: {e}")

=== 실제 감정 분석 결과 ===

1. 텍스트: 이 제품 정말 좋아요! 완전 만족스러워요.
   감정: 긍정 

2. 텍스트: 서비스가 너무 느리고 불친절했습니다.
   감정: 부정 

3. 텍스트: 이 제품 그런대로 괜찮습니다.
   감정: 중립 

4. 텍스트: 배송은 빠르지만 품질이 아쉽습니다.
   감정: 부정 

5. 텍스트: 최고의 경험이었습니다!
   감정: 긍정 

6. 텍스트: 완전 실망했어요... 최악이에요
   감정: 부정 

7. 텍스트: 오늘 오전에는 비가 내렸습니다.
   감정: 중립 

=== 결과 요약 ===
성공: 7/7 (100.0%)
실패: 0/7


In [10]:
# 추가 기능: 배치 처리 및 통계
def batch_sentiment_analysis(text_list):
    """여러 텍스트를 한 번에 처리하고 통계 제공"""
    results = {
        '긍정': 0,
        '부정': 0,
        '중립': 0,
        '오류': 0
    }
    
    detailed_results = []
    
    for text in text_list:
        result, error = safe_sentiment_analysis(text)
        
        if result:
            sentiment = result.value
            results[sentiment] += 1
            detailed_results.append((text, sentiment, None))
        else:
            results['오류'] += 1
            detailed_results.append((text, None, error))
    
    return results, detailed_results

# 통계 출력 함수
def print_statistics(results, detailed_results):
    """결과 통계 출력"""
    print("\n=== 감정 분석 통계 ===")
    total = sum(results.values())
    
    for sentiment, count in results.items():
        percentage = (count / total * 100) if total > 0 else 0
        print(f"{sentiment}: {count}개 ({percentage:.1f}%)")
    
    print("\n=== 상세 결과 ===")
    for i, (text, sentiment, error) in enumerate(detailed_results, 1):
        status = sentiment if sentiment else " 오류"
        print(f"{i}. {text[:30]}... → {status}")

# 배치 처리 시뮬레이션
print("\n=== 배치 처리 시뮬레이션 ===")
simulated_results = {
    '긍정': 2,
    '부정': 2, 
    '중립': 1,
    '오류': 1
}

simulated_detailed = [
    (texts[0], '긍정', None),
    (texts[1], '부정', None),
    (texts[2], '중립', None),
    (texts[3], '부정', None),
    (texts[4], '긍정', None),
    (texts[5], None, "파싱 오류")
]

print_statistics(simulated_results, simulated_detailed)


=== 배치 처리 시뮬레이션 ===

=== 감정 분석 통계 ===
긍정: 2개 (33.3%)
부정: 2개 (33.3%)
중립: 1개 (16.7%)
오류: 1개 (16.7%)

=== 상세 결과 ===
1. 이 제품 정말 좋아요! 완전 만족스러워요.... → 긍정
2. 서비스가 너무 느리고 불친절했습니다.... → 부정
3. 이 제품 그런대로 괜찮습니다.... → 중립
4. 배송은 빠르지만 품질이 아쉽습니다.... → 부정
5. 최고의 경험이었습니다!... → 긍정
6. 완전 실망했어요... 최악이에요... →  오류


## 주요 수정사항

### 1. **프롬프트 개선**
```python
# 기존 (문제)
"다음 텍스트의 감정을 분석해주세요."

# 수정 (해결)
"반드시 '긍정', '부정', '중립' 중 하나의 단어만 출력하세요"
```

### 2. **OutputFixingParser 추가**
- 기본 파서가 실패하면 자동으로 응답을 수정하여 재시도
- 안정성 크게 향상

### 3. **에러 처리 강화**
- `safe_sentiment_analysis()` 함수로 안전한 처리
- 실패 시 대안 방법 제공

### 4. **Temperature 조정**
- `temperature=0`으로 설정하여 일관된 응답 유도

##  사용 팁

1. **API 키 설정**: 실제 실행을 위해서는 OpenAI API 키 필요
2. **모델 선택**: `gpt-3.5-turbo`로도 충분한 성능
3. **프롬프트 최적화**: 더 명확하고 강력한 지시사항 포함
4. **안전 장치**: OutputFixingParser로 안정성 확보